In [1]:
#!/usr/bin/env python3

import pandas as pd
import errno    
import os

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

'''
Author: Enrico Ceccolini
    TODO write the description
'''

datadir = "/datasets/eurora_data/db/"
infile_jobs_to_nodes = datadir + "job_nodes.csv"
infile_jobs = datadir + "jobs.csv"

# settings
interval_comment = "April"
cpu_suffix = "0_1min_"

start_time = pd.to_datetime('2014-04-01')
end_time = pd.to_datetime('2014-05-01')


## notice that 43 doesn't exists in the db
# cpus=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']
cpus=['01'] # test

In [2]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data contains 469095 records


In [12]:
# jobs_to_nodes_whole_data.head(2)

In [11]:
# jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] == 129]

Nodo 129???

In [6]:
# jobs_to_nodes_whole_data.sort_values('node_id')

#### solve the problem of jobs runned on node 129

In [13]:
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129]

In [14]:
df = jobs_to_nodes_whole_data.groupby(['job_id_string','node_id']).size().reset_index(name='counts')
df = df.sort_values('counts')
# df[df['node_id'] == 1]

for example the job 1008449.node129 has multiple entries in the same node

In [15]:
# jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['job_id_string'] == '1008449.node129']

#### solve the problem of duplicates job_id_string, node_id

In [16]:
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])

In [17]:
jobs_to_nodes_whole_data.shape[0]

444610

before 469095

In [18]:
jobs_whole_data = pd.read_csv(infile_jobs)
print("jobs_whole_data contains {} records".format(jobs_whole_data.shape[0]))

jobs_whole_data contains 405771 records


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


how can be? 469095 vs 405771. -> jobs that require more than on node to be executed

In [19]:
jobs_whole_data.head(1)

,job_id,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,time_req,deleted,dependency,exit_status,pbs_exit_code
0,1,498458.node129,rth2e6t0,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,alupi000@node129.eurora.cineca.it,1,16,14,04:00,NaN,NaN,COMPLETED,NaN


In [28]:
# completed_jobs_data = jobs_whole_data[jobs_whole_data.exit_status == 'COMPLETED']
# completed_jobs_data = jobs_whole_data[jobs_whole_data.exit_status != 'RUN']
completed_jobs_data = jobs_whole_data[jobs_whole_data.end_time != '0000-00-00 00:00:00']
print("completed_jobs_data contains {} records".format(completed_jobs_data.shape[0]))

completed_jobs_data contains 404871 records


In [30]:
# join the two dataframe

#merged_jobs_to_nodes = pd.merge(jobs_to_nodes_whole_data, jobs_whole_data, on='job_id_string')
merged_jobs_to_nodes = pd.merge(jobs_to_nodes_whole_data, completed_jobs_data, how='left', on='job_id_string')
# print(merged_jobs_to_nodes.shape)

(444610, 22)


In [31]:
merged_jobs_to_nodes.head(2)

,job2node_id,node_id,job_id_string,ncpus,ngpus,nmics,mem_requested,job_id,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,time_req,deleted,dependency,exit_status,pbs_exit_code
0,1,17,498458.node129,16,0,0,14680064,1.0,rth2e6t0,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,alupi000@node129.eurora.cineca.it,1.0,16.0,14.0,04:00,NaN,NaN,COMPLETED,NaN
1,2,9,498459.node129,16,0,0,14680064,2.0,CPMD,parallel,2014-03-31 00:36:51,2014-03-31 00:36:51,2014-03-31 00:40:13,framondo@node129.eurora.cineca.it,1.0,16.0,14.0,01:00,NaN,NaN,COMPLETED,NaN


drop where node_id is NaN

In [44]:
merged_jobs_to_nodes.groupby('node_id')['job2node_id'].nunique().reset_index(name='counts')

,node_id,counts
0,1,1875
1,2,240
2,3,2388
3,4,2167
4,5,484
5,6,1234
6,7,2065
7,8,1761
8,9,840
9,10,1677


torna l'assenza del node 43 e la scarsità di 22, 41, 53,

In [33]:
#regular_data_jobs = merged_jobs_to_nodes[merged_jobs_to_nodes.node_id >= 1]
#regular_data_jobs = regular_data_jobs[regular_data_jobs.node_id <= 64]
#regular_data_jobs.shape

(444610, 22)

In [34]:
interval_data_jobs = merged_jobs_to_nodes[pd.to_datetime(merged_jobs_to_nodes['start_time']) > start_time]
interval_data_jobs = interval_data_jobs[pd.to_datetime(interval_data_jobs['end_time']) < end_time]
interval_data_jobs = interval_data_jobs.sort_values('run_start_time')
print("interval_data_jobs contains {} records".format(interval_data_jobs.shape[0]))

interval_data_jobs contains 93236 records


In [35]:
# for cpu in cpus:

cpu = '01'
print(cpu)

01


In [36]:
interval_jobs_node_data = interval_data_jobs[interval_data_jobs['node_id'] == int(cpu)]
print("interval_jobs_node_data contains {} records".format(interval_jobs_node_data.shape[0]))

interval_jobs_node_data contains 486 records


In [47]:
infile_node = datadir + "CPUs/" + interval_comment + "/cpu" + cpu + "_" + cpu_suffix + interval_comment
node_data = pd.read_csv(infile_node + ".csv") # Read a comma-separated values (csv) file into DataFrame

(43200, 6)


In [48]:
node_data['active_cores'] = pd.Series()
node_data['active_jobs'] = pd.Series()
node_data.head(5)

,timestamp,01,cpu_id,pow_cpu,pow_dram,pow_pkg,active_cores,active_jobs
0,2014-04-01 00:00:00,1.0,0.0,2.155305,2.507122,17.899565,NaN,NaN
1,2014-04-01 00:01:00,1.0,0.0,2.033380,2.472140,17.762189,NaN,NaN
2,2014-04-01 00:02:00,1.0,0.0,2.037560,2.526941,17.769899,NaN,NaN
3,2014-04-01 00:03:00,1.0,0.0,2.030268,2.487323,17.759689,NaN,NaN
4,2014-04-01 00:04:00,1.0,0.0,1.985083,2.500854,17.683834,NaN,NaN


In [49]:
for index, row in node_data.iterrows():
    # print(row)
    running_jobs = interval_jobs_node_data[pd.to_datetime(interval_jobs_node_data['run_start_time']) <= row['timestamp']]
    running_jobs = running_jobs[pd.to_datetime(running_jobs['end_time']) >= row['timestamp']]
    node_data.loc[index,'active_jobs'] = running_jobs.shape[0]
    node_data.loc[index,'active_cores'] = running_jobs['ncpus'].sum()

outfile_node = infile_node + "_active_cores_and_jobs"
node_data.to_csv(outfile_node + ".csv")

In [41]:
node_data['active_jobs'].max()

7.0

In [45]:
node_data.groupby('active_cores')['timestamp'].nunique().reset_index(name='counts')

,active_cores,counts
0,0.0,38473
1,1.0,639
2,2.0,162
3,3.0,7
4,4.0,103
5,5.0,1
6,6.0,1
7,7.0,6
8,8.0,162
9,12.0,313
